# Data streaming for real-time object-interaction recognition and analytics using HoloLens2

<center> <h2> Introduction </h2></center> 
<p>This project in meant to apply Object Detection on real time data coming from a pair of Microsoft Hololens2's smartglasses.</p>

<p><a href=https://github.com/marco-ardi/Data-Streaming-Object-Detector-Pipeline>Repository Here </a></p>

<center> <h2> Prerequisities </h2></center>

* Linux, MacOs or Windows Subsistem for Linux (WSL2)
* Nvidia GPU and CUDA drivers. [Guide on WSL2](https://ubuntu.com/blog/getting-started-with-cuda-on-ubuntu-on-wsl-2)
* Python version>= 3.6

<center> <h2> Microsoft Hololens2 </h2></center>
<center>
    
<img src="Images/hololens_img.jpg" width="30%" height="30%" >
</center>
<p style="float: center; padding-top: 50px; ">
Microsoft Hololens2 is a pair of smartglasses for Augmented Reality: it has several cameras (up to 2K resolution) and sensors, it can detect Hands, Eye Gaze, Voice and can also do spatial mapping, making it perfect as a Data Source in a Data Science Project!


<center>
    
<img src="Images/meme0 - hot.jpg" width="30%" height="30%">
</center>

<center> <h2> Data Pipeline </h2></center>
<img style="float: center;" src="Images/pipelinev4.png" >




Here's the Data Pipeline:
<ul style="float: center; padding-left: 80px;">
  <li>Data Source: Microsoft Hololens2</li>
  <li>Data Ingestion: Logstash</li>
  <li>Data Streaming: Kafka + KafkaUI</li>
  <li>Data Processing: Spark</li>
  <li>Data Storage: Elastis Search</li>
  <li>Data Visualization: Kibana</li>
</ul>

<center> <h2> Data Source: Microsoft Hololens2 </h2></center>




<center>

<img src="Images/meme1 - minecraft.jpg" width="30%" height="30%">
</center>

Hololens2 is a great data source: It is used to retrieve data about hand and eye gaze location and photos.
A client socket is opened in Hololens and sends, once per 2 seconds, a photo and a formatted string which contains hand and eye location.


On the other side, a server socket receives and decodes photo and logs, storing them in a .csv file

In [ ]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    with conn:
        print('Connected by', addr)
        while True:
            data = conn.recv(1024*8)
            stringdata = data.decode('utf-8')
            if not data: continue  

            received_data = []
            received_data = stringdata.split(' ')

            with open(CSV_PATH, 'a', encoding='UTF8', newline='') as myfile:
                wr = csv.writer(myfile, quoting=csv.QUOTE_NONE, escapechar=' ')
                wr.writerow(received_data)

            img = conn.recv(160000) #worst case scenario for a 1280x720 jpeg image is 160000 bytes
            if not img: continue         
            try:
                file_bytes = np.asarray(bytearray(io.BytesIO(img).read()), dtype=np.uint8)
                imgToShow = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
                cv2.imwrite(IMGS_PATH+ received_data[0] +'.jpg', imgToShow)


<center> <h2> Data Ingestion: Logstash </h2></center>
<center>

<img src="Images/logstashlogo.png">
</center>

Logstash is an open source data collection engine with real-time pipelining capabilities. It can dynamically unity data from different sources and normalize the data into destination of your choice.

It is used for getting data from the server socket and prepare them for Kafka.

In [ ]:
input{ 
    file{
        path=>"/usr/share/logstash/csv/logs2D.csv"
        start_position=>"beginning"
    }
}
filter{
    csv{
        separator=>","
        columns=>["id", "c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9", "c10", "c11", "c12", "c13", "c14", "c15", "c16", "c17", "c18", "c19", "c20", "c21", "c22"]
    }  
    mutate{
        remove_field => ["@timestamp", "@version", "host", "path", "message"]
    }
}
output{
    kafka {
        codec => json
        topic_id => "tap"
        bootstrap_servers => "kafkaserver:9092"
    }
    stdout {}
}

<center> <h2> Data Streaming: Kafka </h2></center>
<center>

<img  src="Images/kafkalogo.png">
</center>

<center>

<img src="Images/meme2 - kafka.jpg" width="30%" height="30%">
</center>

Apache Kafka is an open-source software which provides a framework for storing, reading and analysing streaming data. It is used for handling real time data feeds with low-latency and high-throughput.

It is used for receiving logs from Logstash, storing them and sending them to Spark for doing object detection.

<center> <h2> Data Processing: Spark </h2></center>
<center>

<img src="Images/sparklogo.png">
</center>

<center>

<img  src="Images/meme3 - sparta-spark.jpg">
</center>

Apache Spark is an open-source unified analytics engine for large-scale data processing. It allows to program in Java, R, Scala and Python, ensuring parallelism and fault tolerance.

It is used for doing object detection to the stream coming from Kafka.

<center> <h3> Detectron2 </h3></center>
<center>
    
<img src="Images/detectronimage.png" width="50%" height="50%">
</center>

Written in Python, Detectron2 uses deep learning for object detection, providing a easy-to-use and extremely powerful framework for doing detection on photos or videos.

<center>

<img src="Images/mymeme1.jpg" width="30%" height="30%">
</center>

<center>

<img src="Images/mymeme2.jpg" width="30%" height="30%">
</center>

<center> <h2> Data Storage: Elastic Search </h2></center>
<center>
    
<img src="Images/elasticsearchlogo.png">
</center>

Elastic Search is a distributed, free and open search and analytics engine for all types of data.

It stores object detection information from Spark, then passes them to Kibana for data visualization.

<center> <h2> Data Visualization: Kibana </h2></center>
<center>
    
<img src="Images/kibanalogo2.png">
</center>

Kibana is a data visualization and exploration tool used for log and time-series analytics, application monitoring, and operational intelligence use cases.

It offers powerful and easy-to-use features such as histograms, line graphs, pie charts, heat maps, and built-in geospatial support.

<center> <h2> Live Demo! </h2></center>
<center>
    
<img src="Images/memex - live demo.jpg">
</center>

<center>

<img src="Images/mymeme3.jpg" width="40%" height="40%">
</center>

Live demo:
<ul style="float: center; padding-left: 80px;">
  <li>start server.py (checking IP address)</li>
  <li>start stringConverter.py</li>
  <li>start Hololens' script</li>
  <li>docker-compose up</li>
  <li>Enjoy!</li>
</ul>


<h3>Realized By Marco Ardizzone X81001077</h3>
<h4>Special Thanks to Simone Benedetto</h4>